In [1]:
import pandas as pd
import numpy as np

# paths and filenames

In [109]:
path_orgtree = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/2.4 orgtree/"

In [110]:
filein_orgtree = "orgtree_4_2.xlsx"

In [172]:
fileout_orgtree = "orgtree_4_3_jeongsu.xlsx"

In [112]:
orgtree = pd.read_excel(path_orgtree+filein_orgtree)
orgtree.shape

(1290, 8)

In [113]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToCurrent_PrimaryInstitution',
       'LinkToCurrent_OrgName', 'Notes'],
      dtype='object')

# clean Link fields

In [114]:
# replace NAN with Deprecated
orgtree[orgtree["LinkToCurrent_OrgName"]=="NAN"]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,Notes
797,1.0,내각,수매양정성,NaN,NaN,NAN,NAN,NaN
806,1.0,내각,외무성_X국_대련무역대표부,NaN,NaN,NAN,NAN,NaN
808,1.0,내각,외무성_X국_몰타대사관,NaN,NaN,NAN,NAN,NaN
903,1.0,내각,일용품공업성,NaN,NaN,NAN,NAN,NaN
970,1.0,노동당,당중앙위원회_경제정책검열부,NaN,NaN,NAN,NAN,NaN
1004,1.0,노동당,당중앙위원회_신소실,중앙위원회,NaN,NAN,NAN,NaN
1086,1.0,정무원,원유공업부,NaN,NaN,NAN,NAN,NaN


In [115]:
orgtree.loc[orgtree["LinkToCurrent_PrimaryInstitution"]=="NAN","LinkToCurrent_PrimaryInstitution"] = "Deprecated"

In [116]:
orgtree.loc[orgtree["LinkToCurrent_OrgName"]=="NAN","LinkToCurrent_OrgName"] = "Deprecated"

In [117]:
orgtree[orgtree["LinkToCurrent_PrimaryInstitution"]=="NAN"]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,Notes


In [118]:
orgtree[orgtree["LinkToCurrent_OrgName"]=="NAN"]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,Notes


In [119]:
# replace LinkToCurrent_PrimaryInstitution==nan with "Please_Revise"

In [120]:
orgtree.loc[orgtree["LinkToCurrent_PrimaryInstitution"].isna(),"LinkToCurrent_PrimaryInstitution"] = "Please_Revise"

In [173]:
orgtree[orgtree["LinkToCurrent_PrimaryInstitution"].isna()]

KeyError: 'LinkToCurrent_PrimaryInstitution'

In [122]:
# change and add link columns

In [123]:
orgtree["LinkToNext_Year"]=np.nan

In [124]:
orgtree.rename(columns={"LinkToCurrent_PrimaryInstitution":"LinkToNext_PI","LinkToCurrent_OrgName":"LinkToNext_Org"},inplace=True)
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org', 'Notes',
       'LinkToNext_Year'],
      dtype='object')

In [125]:
columns_orgtree = ['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org','LinkToNext_Year','Notes']

In [126]:
orgtree = orgtree[columns_orgtree]
orgtree.shape

(1290, 9)

In [127]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

# add LinkToNext_PI & LinkToNext_Org as Primary Key

In [128]:
orgtree_links = orgtree[['InsideGov','LinkToNext_PI', 'LinkToNext_Org']]

In [129]:
merged = orgtree.merge(orgtree_links,how="outer",
                        left_on=['PrimaryInstitution', 'OrgName'],
                        right_on=['LinkToNext_PI', 'LinkToNext_Org'],
                       indicator=True)

In [130]:
merged["_merge"].unique()

['left_only', 'both', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [131]:
merged[merged["_merge"]=="both"].shape

(245, 13)

In [132]:
merged[merged["_merge"]=="left_only"].shape

(1152, 13)

In [133]:
merged[merged["_merge"]=="right_only"].shape

(1045, 13)

In [134]:
merged_right = merged[merged["_merge"]=="right_only"]
merged_right.shape

(1045, 13)

In [135]:
merged_right.columns

Index(['InsideGov_x', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI_x', 'LinkToNext_Org_x',
       'LinkToNext_Year', 'Notes', 'InsideGov_y', 'LinkToNext_PI_y',
       'LinkToNext_Org_y', '_merge'],
      dtype='object')

In [136]:
# 37 [LinkToNext_PI, LinkToNext_Org] not contained in [PI, Org]
merged_right = merged_right.drop_duplicates()
merged_right.shape

(36, 13)

In [137]:
merged_right

,InsideGov_x,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI_x,LinkToNext_Org_x,LinkToNext_Year,Notes,InsideGov_y,LinkToNext_PI_y,LinkToNext_Org_y,_merge
1397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Current,Current,right_only
1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Current,Current,right_only
2366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,평양예술단,NaN,right_only
2367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Please_Revise,Please_Revise,right_only
2374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Please_Revise,Please_Revise,right_only
2376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,내각,교육위원회_고등교육성_계응상사리원농업대학,right_only
2377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Uncertain,Uncertain,right_only
2380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Uncertain,Uncertain,right_only
2397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Please_Revise,NaN,right_only
2399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Please_Revise,NaN,right_only


In [138]:
# prep merged_right columns for concat with orgtree
columns_select = ['InsideGov_y','LinkToNext_PI_y','LinkToNext_Org_y',
                  'ImmediateSuperiorOrg','SupervisesParent',
                  'LinkToNext_PI_x','LinkToNext_Org_x','LinkToNext_Year','Notes']

In [139]:
merged_right = merged_right[columns_select]
merged_right.shape

(36, 9)

In [140]:
merged_right.columns = orgtree.columns

In [141]:
merged_right

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
1397,0.0,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN
1850,1.0,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN
2366,0.0,평양예술단,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2367,0.0,Please_Revise,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN
2374,1.0,Please_Revise,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN
2376,0.0,내각,교육위원회_고등교육성_계응상사리원농업대학,NaN,NaN,NaN,NaN,NaN,NaN
2377,0.0,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN
2380,1.0,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN
2397,0.0,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2399,1.0,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
# concatenate orgtree & merged_right
orgtree = pd.concat([orgtree,merged_right])

In [143]:
orgtree.shape

(1326, 9)

In [144]:
# Step 1. drop NA keys
orgtree.dropna(subset=["PrimaryInstitution","OrgName"],how="all",inplace=True)
orgtree.shape

(1326, 9)

In [145]:
# Step 2. verify no duplicate rows
orgtree[orgtree.duplicated()]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes


In [146]:
# Step 2. verify no duplicate keys
orgtree.sort_values(["PrimaryInstitution","OrgName","InsideGov"],inplace=True)
orgtree[orgtree.duplicated(["PrimaryInstitution","OrgName"],keep=False)]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
1397,0.0,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN
1850,1.0,Current,Current,NaN,NaN,NaN,NaN,NaN,NaN
2420,1.0,Deprecated,Deprecated,NaN,NaN,NaN,NaN,NaN,NaN
2427,NaN,Deprecated,Deprecated,NaN,NaN,NaN,NaN,NaN,NaN
2367,0.0,Please_Revise,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN
2374,1.0,Please_Revise,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN
2397,0.0,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2399,1.0,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2408,NaN,Please_Revise,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2377,0.0,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
# drop all of the above duplicates
orgtree.drop_duplicates(["PrimaryInstitution","OrgName"],keep=False,inplace=True)
orgtree.shape

(1315, 9)

# recursively add ImmedSuperiorOrgs, drop duplicates and sort

In [148]:
orgtree.sort_values(["InsideGov","PrimaryInstitution","OrgName"],ascending=[False,True,True],inplace=True)

In [149]:
orgtree.shape

(1315, 9)

In [150]:
orgtree[0:2]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
537,1.0,NAN,인민군,NaN,NaN,Please_Revise,Please_Revise,NaN,NaN
538,1.0,국무위원회,국방성_참모부_총무국,국방성,NaN,Current,Current,NaN,NaN


In [151]:
# orgtree with non-NA OrgName
orgtree_proc = orgtree[~orgtree["OrgName"].isna()]
orgtree_proc.shape

(1045, 9)

In [152]:
orgtree_2order = orgtree_proc.loc[orgtree_proc.OrgName.str.contains("_"),["InsideGov","PrimaryInstitution","OrgName"]]
orgtree_2order["OrgName"] = orgtree_2order["OrgName"].apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_2order.drop_duplicates(inplace=True)
orgtree_2order.shape

(186, 3)

In [153]:
orgtree_3order = orgtree_2order.loc[orgtree_2order.OrgName.str.contains("_"),["InsideGov","PrimaryInstitution","OrgName"]]
orgtree_3order["OrgName"] = orgtree_3order["OrgName"].apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_3order.drop_duplicates(inplace=True)
orgtree_3order.shape

(31, 3)

In [154]:
orgtree_4order = orgtree_3order.loc[orgtree_3order.OrgName.str.contains("_"),["InsideGov","PrimaryInstitution","OrgName"]]
orgtree_4order.OrgName = orgtree_4order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_4order.drop_duplicates(inplace=True)
orgtree_4order.shape

(9, 3)

In [155]:
orgtree_5order = orgtree_4order.loc[orgtree_4order.OrgName.str.contains("_"),["InsideGov","PrimaryInstitution","OrgName"]]
orgtree_5order.OrgName = orgtree_5order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_5order.drop_duplicates(inplace=True)
orgtree_5order.shape

(1, 3)

In [156]:
orgtree_5order

,InsideGov,PrimaryInstitution,OrgName
1131,1.0,총참모부,작전총국


In [157]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

In [158]:
orgtree_higher_order = pd.concat([orgtree_2order,orgtree_3order,orgtree_4order])
orgtree_higher_order.drop_duplicates(inplace=True)
orgtree_higher_order.sort_values(["InsideGov","PrimaryInstitution","OrgName"],ascending=False,inplace=True)
orgtree_higher_order.shape

(191, 3)

In [159]:
# find new orgtree_higher_order whose PrimaryOrg & OrgName are not contained in orgtree
orgtree_higher_order_merged = orgtree_higher_order.merge(orgtree[["PrimaryInstitution","OrgName"]],
                                             how="left",on=["PrimaryInstitution","OrgName"],
                                             suffixes=("","_right"),indicator=True)
orgtree_higher_order_unique = orgtree_higher_order_merged[orgtree_higher_order_merged["_merge"]=="left_only"]
orgtree_higher_order_unique.shape

(52, 4)

In [160]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

In [161]:
orgtree_higher_order_unique.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', '_merge'], dtype='object')

In [162]:
# drop _merge column in orgtree_higher_order_unique
orgtree_higher_order_unique = orgtree_higher_order_unique.drop(columns=["_merge"],axis=1)
orgtree_higher_order_unique.shape

(52, 3)

In [163]:
# add ImmediateSuperiorOrg to orgtree_higher_order_unique
orgtree_higher_order_unique["ImmediateSuperiorOrg"] = orgtree_higher_order_unique["OrgName"].apply(lambda row: "_".join(row.split("_")[:-1]))

In [164]:
orgtree.columns

Index(['InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'SupervisesParent', 'LinkToNext_PI', 'LinkToNext_Org',
       'LinkToNext_Year', 'Notes'],
      dtype='object')

In [165]:
# add columns [LinkToNext_PI, LinkToNext_Org, Notes] and set them to null
orgtree_higher_order_unique["LinkToNext_PI"] = "Please_Revise"
orgtree_higher_order_unique["LinkToNext_Org"] = "Please_Revise"
orgtree_higher_order_unique["LinkToNext_Year"] = "Please_Revise"
orgtree_higher_order_unique["Notes"] = np.nan

In [166]:
# concatenate orgtree and orgtree_higher_order_unique
orgtree_concat = pd.concat([orgtree,orgtree_higher_order_unique])
orgtree_concat.shape

(1367, 9)

In [167]:
# confirm no duplicate (PrimaryInstitution, OrgName) pairs 
orgtree_concat[orgtree_concat.duplicated(["PrimaryInstitution","OrgName"],keep=False)]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes


In [168]:
# sort
orgtree_concat.sort_values(["InsideGov","PrimaryInstitution","OrgName"],ascending=False,inplace=True,ignore_index=True)

In [169]:
orgtree_concat.shape

(1367, 9)

In [170]:
orgtree_concat[0:5]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes
0,1.0,호위사령부,호위총국_평양경비사령부,호위사령부,0.0,Current,Current,NaN,NaN
1,1.0,호위사령부,호위총국_제55처,호위총국,0.0,Current,Current,NaN,전시 지휘소
2,1.0,호위사령부,호위총국_제4국,호위총국,0.0,Current,Current,NaN,NaN
3,1.0,호위사령부,호위총국_제2국,호위총국,0.0,Current,Current,NaN,NaN
4,1.0,호위사령부,호위총국_제1국_행사안전부,호위총국_제1국,0.0,Current,Current,NaN,NaN


# Please_Revise null InsideGov and LinkToNext_

In [182]:
orgtree.loc[orgtree["LinkToNext_PI"].isna()].shape

(0, 9)

In [175]:
orgtree.loc[orgtree["LinkToNext_PI"].isna(),"LinkToNext_PI"] = "Please_Revise"

In [181]:
orgtree.loc[orgtree["LinkToNext_Org"].isna()].shape

(0, 9)

In [177]:
orgtree.loc[orgtree["LinkToNext_Org"].isna(),"LinkToNext_Org"] = "Please_Revise"

In [180]:
orgtree.loc[orgtree["LinkToNext_Year"].isna()].shape

(0, 9)

In [179]:
orgtree.loc[orgtree["LinkToNext_Year"].isna(),"LinkToNext_Year"] = "Please_Revise"

In [187]:
orgtree.loc[orgtree["LinkToNext_PI"]=="Current","LinkToNext_Year"] = "Current"

In [185]:
orgtree.loc[orgtree["InsideGov"].isna()].shape

(0, 9)

In [184]:
orgtree.loc[orgtree["InsideGov"].isna(),"InsideGov"] = "Please_Revise"

# export orgtree

In [189]:
# orgtree.to_excel(path_orgtree+fileout_orgtree,index=False)

# double-check jol & orgtree

In [200]:
# confirmed all jol key are in orgtree!

# all exceptions are "Uncertain"

In [191]:
path_joborglink = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/2.3 joborglink/"

In [192]:
fileout_joborglink = "joborglink_7_2.xlsx"

In [193]:
jol = pd.read_excel(path_joborglink+fileout_joborglink)

In [194]:
merge = orgtree.merge(jol,how="outer",on=["PrimaryInstitution","OrgName"],indicator=True)

In [196]:
merge[merge["_merge"]=="both"].shape

(1417, 12)

In [197]:
merge[merge["_merge"]=="left_only"].shape

(620, 12)

In [198]:
merge[merge["_merge"]=="right_only"].shape

(9, 12)

In [199]:
merge[merge["_merge"]=="right_only"]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,SupervisesParent,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes_x,OrgString,Notes_y,_merge
2037,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,노동당 기계공업협회,기계공업부; 비고: 당중앙위원회?,right_only
2038,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,노동당 중공업부,중앙위원회->중앙인민위원회; 비고: 중앙인민위원회 is a PrimaryInstit...,right_only
2039,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,노동당 중앙인민위원회 중공업부 제1부,1부 Not found; 비고: 중앙인민위원회 is a PrimaryInstitut...,right_only
2040,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,당 기계공업부,NaN,right_only
2041,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,당 중공업부,NaN,right_only
2042,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,전자자동화공업위원회,NaN,right_only
2043,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,전자자동화공업위원회 기술국,기술국 Not found; 비고: PrimaryInstitution?,right_only
2044,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,전자자동화공업위원회 제1부,1부 Not found; 비고: PrimaryInstitution?,right_only
2045,NaN,Uncertain,Uncertain,NaN,NaN,NaN,NaN,NaN,NaN,제2집단군,NaN,right_only
